In [8]:
import sys
import os
import re

def main():
    file_to_read= 'amazon_cells_labelled.txt'
    positives=[] #list for positive sentences
    negatives=[] #list for negative sentences
    positive_counter=0 #keep track of number of positive sentences
    negative_counter=0 #keep track of number of negative sentences
    
    #opening file and putting unique words into vocabulary set
    with open (file_to_read) as file:
        text=file.read()
        words = sorted(set(re.split(r'\W+', text)))
        words=[word.lower()for word in words]
        length=len(words)
        print("Length of vocabulary set is:", length)
    file.close()
    
    #opening file and placing positive words in positive list and negative list respectively     
    with open (file_to_read, 'r+') as file:
        for line in file:
            if (line.find('1'))!=-1:
                positive_counter +=1
                positives.append(line)
                #print(positives)
            
            elif (line.find('0'))!=-1:
                negative_counter+=1
                negatives.append(line)
                #print(negatives)
                
        print ("Total Number of positive sentences are: ", positive_counter)
        print ("Total Number of negative sentences are: ", negative_counter)
    file.close()

In [9]:
main()

Length of vocabulary set is: 2191
Total Number of positive sentences are:  514
Total Number of negative sentences are:  486
